<a href="https://colab.research.google.com/github/anjalijadhav002/Zeotap_Datascience_Task/blob/main/Anjali_Jadhav_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


customers = pd.read_csv("/content/Customers.csv")
products = pd.read_csv("/content/Products.csv")
transactions = pd.read_csv("/content/Transactions.csv")


transactions_products = transactions.merge(products, on="ProductID")
customer_transactions = transactions_products.merge(customers, on="CustomerID")


customer_profile = customer_transactions.groupby(['CustomerID', 'ProductID']).agg({
    'Quantity': 'sum',
    'TotalValue': 'sum'
}).reset_index()


pivot_table = customer_profile.pivot_table(
    index='CustomerID',
    columns='ProductID',
    values='Quantity',
    fill_value=0
)


scaler = StandardScaler()
pivot_normalized = scaler.fit_transform(pivot_table)
pivot_normalized = pd.DataFrame(pivot_normalized, index=pivot_table.index, columns=pivot_table.columns)


similarity_matrix = cosine_similarity(pivot_normalized)
similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.index, columns=pivot_table.index)


def get_top_3_lookalikes(customer_id):
    if customer_id in similarity_df.index:
        similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:4]  # Exclude self
        return list(zip(similar_customers.index, similar_customers.values))
    return []


lookalike_results = {}
for customer in customers['CustomerID'][:20]:
    lookalike_results[customer] = get_top_3_lookalikes(customer)


lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index', columns=['Top_1', 'Top_2', 'Top_3'])

lookalike_df.to_csv("Anjali_Jadhav_Lookalike.csv", index_label="CustomerID")
print("Lookalike results saved to Anjali_Jadhav_Lookalike.csv.")

Lookalike results saved to Anjali_Jadhav_Lookalike.csv.
